In [1]:
# Import required libraries
import numpy as np
import pandas as pd
import os
import requests
import hvplot.pandas
from pathlib import Path
%matplotlib inline
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.datasets import make_blobs,make_moons
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
import tensorflow as tf
from sklearn.metrics import r2_score
from datetime import date
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

2024-02-27 20:10:30.235293: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
# Read in any csv files
funds_df = pd.read_csv(Path('bill-rates-2002-2023.csv'),
                           index_col='Date',
                           parse_dates=True,
                      infer_datetime_format=True)
funds_df = funds_df.dropna()   
funds_df

,4 WEEKS BANK DISCOUNT,4 WEEKS COUPON EQUIVALENT,8 WEEKS BANK DISCOUNT,8 WEEKS COUPON EQUIVALENT,13 WEEKS BANK DISCOUNT,13 WEEKS COUPON EQUIVALENT,17 WEEKS BANK DISCOUNT,17 WEEKS COUPON EQUIVALENT,26 WEEKS BANK DISCOUNT,26 WEEKS COUPON EQUIVALENT,52 WEEKS BANK DISCOUNT,52 WEEKS COUPON EQUIVALENT
Date,,,,,,,,,,,,
2023-12-29,5.33,5.44,5.33,5.46,5.20,5.35,5.21,5.39,5.05,5.26,4.55,4.79
2023-12-28,5.31,5.42,5.29,5.42,5.25,5.41,5.22,5.40,5.06,5.28,4.58,4.82
2023-12-27,5.28,5.39,5.29,5.42,5.24,5.40,5.21,5.39,5.05,5.27,4.54,4.78
2023-12-26,5.27,5.38,5.27,5.40,5.25,5.41,5.22,5.40,5.06,5.28,4.58,4.82
2023-12-22,5.27,5.38,5.26,5.39,5.24,5.39,5.22,5.40,5.10,5.32,4.62,4.86
...,...,...,...,...,...,...,...,...,...,...,...,...
2022-10-25,3.40,3.46,3.62,3.69,3.97,4.07,4.12,4.23,4.36,4.52,4.40,4.61
2022-10-24,3.42,3.48,3.65,3.72,4.00,4.10,4.14,4.26,4.38,4.54,4.40,4.61
2022-10-21,3.42,3.48,3.64,3.71,3.90,3.99,4.13,4.25,4.29,4.44,4.38,4.59


In [3]:
# Import API data
from dotenv import load_dotenv
import alpaca_trade_api as tradeapi

In [4]:
#3
# Load .env enviroment variables/
load_dotenv('alpaca_keys.env')

True

In [5]:
#4
# Set the tickers
tickers = ['SPY','EWA','EWC','EWJ','EWU','EWZ','EWW','EZU','IWM','MCHI']

# Create the shares DataFrame
df_etf = pd.DataFrame(index=tickers)

In [6]:
#5
# Set Alpaca API key and secret
alpaca_api_key = os.getenv('alpaca_key')
alpaca_secret_key = os.getenv('alpaca_secret_key')

# Verify that Alpaca key and secret were correctly loaded
print(f"Alpaca Key type: {type(alpaca_api_key)}")
print(f"Alpaca Secret Key type: {type(alpaca_secret_key)}")

Alpaca Key type: <class 'str'>
Alpaca Secret Key type: <class 'str'>


In [7]:
#6
# Create the Alpaca API object
alpaca = tradeapi.REST(
    alpaca_api_key,
    alpaca_secret_key,
    api_version="v2")

In [8]:
#6a
# Create current date variable
import datetime
today = datetime.date.today()
historical = today - datetime.timedelta(days=550)

In [9]:
#6b
# Format current date as ISO format
# Set start and end datetimes of 18 months
start_date = pd.Timestamp(historical, tz="America/New_York").isoformat()
end_date = pd.Timestamp(today, tz="America/New_York").isoformat()

In [10]:
#7
# Set timeframe to 1Day for the Alpaca API
timeframe = "1Day"


In [11]:
#8
# Get 18 months of pricing data for ETF portfolio
df_portfolio = alpaca.get_bars(
    tickers,
    timeframe,
    start=start_date,
    end=end_date).df

df_portfolio

,close,high,low,trade_count,open,volume,vwap,symbol
timestamp,,,,,,,,
2022-08-26 04:00:00+00:00,22.44,23.140,22.410,9323,23.11,4546397,22.738378,EWA
2022-08-29 04:00:00+00:00,22.36,22.460,22.265,8311,22.35,3117618,22.380320,EWA
2022-08-30 04:00:00+00:00,22.05,22.490,22.010,9813,22.49,4363121,22.153321,EWA
2022-08-31 04:00:00+00:00,21.91,22.190,21.910,8650,22.14,3948736,22.029738,EWA
2022-09-01 04:00:00+00:00,21.73,21.805,21.530,6800,21.78,3012913,21.645007,EWA
...,...,...,...,...,...,...,...,...
2024-02-21 05:00:00+00:00,497.21,497.370,493.560,499405,495.42,59603771,495.831166,SPY
2024-02-22 05:00:00+00:00,507.50,508.490,503.020,613961,504.01,76402535,505.832024,SPY
2024-02-23 05:00:00+00:00,507.85,510.130,507.100,489047,509.27,61309016,508.438618,SPY


In [12]:
# Drop time from index
df_portfolio.index = df_portfolio.index.date
df_portfolio

,close,high,low,trade_count,open,volume,vwap,symbol
2022-08-26,22.44,23.140,22.410,9323,23.11,4546397,22.738378,EWA
2022-08-29,22.36,22.460,22.265,8311,22.35,3117618,22.380320,EWA
2022-08-30,22.05,22.490,22.010,9813,22.49,4363121,22.153321,EWA
2022-08-31,21.91,22.190,21.910,8650,22.14,3948736,22.029738,EWA
2022-09-01,21.73,21.805,21.530,6800,21.78,3012913,21.645007,EWA
...,...,...,...,...,...,...,...,...
2024-02-21,497.21,497.370,493.560,499405,495.42,59603771,495.831166,SPY
2024-02-22,507.50,508.490,503.020,613961,504.01,76402535,505.832024,SPY
2024-02-23,507.85,510.130,507.100,489047,509.27,61309016,508.438618,SPY
2024-02-26,505.99,508.750,505.860,433719,508.30,50353919,507.080345,SPY


In [13]:
# Separate ticker data
SPY = df_portfolio[df_portfolio['symbol']=='SPY'].drop('symbol',axis=1)
EWA = df_portfolio[df_portfolio['symbol']=='EWA'].drop('symbol',axis=1)
EWC = df_portfolio[df_portfolio['symbol']=='EWC'].drop('symbol',axis=1)
EWJ = df_portfolio[df_portfolio['symbol']=='EWJ'].drop('symbol',axis=1)
EWU = df_portfolio[df_portfolio['symbol']=='EWU'].drop('symbol',axis=1)
EWZ = df_portfolio[df_portfolio['symbol']=='EWZ'].drop('symbol',axis=1)
EWW = df_portfolio[df_portfolio['symbol']=='EWW'].drop('symbol',axis=1)
EZU = df_portfolio[df_portfolio['symbol']=='EZU'].drop('symbol',axis=1)
IWM = df_portfolio[df_portfolio['symbol']=='IWM'].drop('symbol',axis=1)
MCHI = df_portfolio[df_portfolio['symbol']=='MCHI'].drop('symbol',axis=1)

In [14]:
# Concatenate tickers
ticker_portfolio = pd.concat([
    SPY,EWA,EWC,EWJ,EWU,EWZ,EWW,EZU,IWM,MCHI],axis=1,keys=[
    'SPY','EWA','EWC','EWJ','EWU','EWZ','EWW','EZU','IWM','MCHI'])
ticker_portfolio                                                           

SPY                                                   \
             close    high       low trade_count    open     volume   
2022-08-26  405.31  419.96  405.2500      669022  419.39  105714602   
2022-08-29  402.63  405.84  401.1999      490122  402.20   66499372   
2022-08-30  398.21  404.10  396.0000      610728  403.85   87190352   
2022-08-31  395.18  401.24  395.0400      504194  399.93   77635142   
2022-09-01  396.42  396.78  390.0400      597633  392.89   80204169   
...            ...     ...       ...         ...     ...        ...   
2024-02-21  497.21  497.37  493.5600      499405  495.42   59603771   
2024-02-22  507.50  508.49  503.0200      613961  504.01   76402535   
2024-02-23  507.85  510.13  507.1000      489047  509.27   61309016   
2024-02-26  505.99  508.75  505.8600      433719  508.30   50353919   
2024-02-27  506.93  507.16  504.7500      394547  506.70   49041173   

                          EWA                    ...     IWM            \
                  vwap  close     high      low  ...    open    volume   
2022-08-26  410.671909  22.44  23.1400  22.4100  ...  195.34  24682950   
2022-08-29  403.536027  22.36  22.4600  22.2650  ...  186.95  20015752   
2022-08-30  399.076591  22.05  22.4900  22.0100  ...  187.82  22928104   
2022-08-31  397.187287  21.91  22.1900  21.9100  ...  184.94  22457303   
2022-09-01  393.107274  21.73  21.8050  21.5300  ...  181.78  31305176   
...                ...    ...      ...      ...  ...     ...       ...   
2024-02-21  495.831166  23.63  23.6900  23.5200  ...  197.66  28821470   
2024-02-22  505.832024  23.86  23.8750  23.7105  ...  198.65  38184703   
2024-02-23  508.438618  23.91  23.9679  23.8700  ...  199.60  37656936   
2024-02-26  507.080345  23.83  23.9050  23.7550  ...  199.69  29063352   
2024-02-27  506.253617  23.99  24.0100  23.9200  ...  203.08  32052440   

                         MCHI                                                \
                  vwap  close     high     low trade_count    open   volume   
2022-08-26  191.039871  50.38  51.9300  50.300       48695  51.815  8188776   
2022-08-29  187.767002  49.95  50.8650  49.930       48971  50.270  7306766   
2022-08-30  185.039434  48.79  49.7600  48.585       33010  49.650  5234435   
2022-08-31  184.087820  49.69  50.3250  49.530       35040  49.790  6156630   
2022-09-01  180.374139  49.22  49.3300  48.680       38620  49.150  5232575   
...                ...    ...      ...     ...         ...     ...      ...   
2024-02-21  197.569559  39.28  39.6150  39.120       18689  39.290  3192627   
2024-02-22  199.066619  39.77  39.8700  39.455       21611  39.810  3510076   
2024-02-23  199.850984  39.90  40.1900  39.730       20980  40.110  6138575   
2024-02-26  200.939981  39.55  39.8607  39.540       24944  39.670  7433109   
2024-02-27  203.566217  40.09  40.2000  40.015       23211  40.080  7255528   

                       
                 vwap  
2022-08-26  50.675660  
2022-08-29  50.371550  
2022-08-30  48.894582  
2022-08-31  49.787465  
2022-09-01  48.959277  
...               ...  
2024-02-21  39.293091  
2024-02-22  39.681310  
2024-02-23  39.897140  
2024-02-26  39.616831  
2024-02-27  40.099680  

[377 rows x 70 columns]

In [16]:
# Drop timestamp from index
#ticker_portfolio.index = ticker_portfolio.index.date
ticker_portfolio

SPY                                                   \
             close    high       low trade_count    open     volume   
2022-08-26  405.31  419.96  405.2500      669022  419.39  105714602   
2022-08-29  402.63  405.84  401.1999      490122  402.20   66499372   
2022-08-30  398.21  404.10  396.0000      610728  403.85   87190352   
2022-08-31  395.18  401.24  395.0400      504194  399.93   77635142   
2022-09-01  396.42  396.78  390.0400      597633  392.89   80204169   
...            ...     ...       ...         ...     ...        ...   
2024-02-21  497.21  497.37  493.5600      499405  495.42   59603771   
2024-02-22  507.50  508.49  503.0200      613961  504.01   76402535   
2024-02-23  507.85  510.13  507.1000      489047  509.27   61309016   
2024-02-26  505.99  508.75  505.8600      433719  508.30   50353919   
2024-02-27  506.93  507.16  504.7500      394547  506.70   49041173   

                          EWA                    ...     IWM            \
                  vwap  close     high      low  ...    open    volume   
2022-08-26  410.671909  22.44  23.1400  22.4100  ...  195.34  24682950   
2022-08-29  403.536027  22.36  22.4600  22.2650  ...  186.95  20015752   
2022-08-30  399.076591  22.05  22.4900  22.0100  ...  187.82  22928104   
2022-08-31  397.187287  21.91  22.1900  21.9100  ...  184.94  22457303   
2022-09-01  393.107274  21.73  21.8050  21.5300  ...  181.78  31305176   
...                ...    ...      ...      ...  ...     ...       ...   
2024-02-21  495.831166  23.63  23.6900  23.5200  ...  197.66  28821470   
2024-02-22  505.832024  23.86  23.8750  23.7105  ...  198.65  38184703   
2024-02-23  508.438618  23.91  23.9679  23.8700  ...  199.60  37656936   
2024-02-26  507.080345  23.83  23.9050  23.7550  ...  199.69  29063352   
2024-02-27  506.253617  23.99  24.0100  23.9200  ...  203.08  32052440   

                         MCHI                                                \
                  vwap  close     high     low trade_count    open   volume   
2022-08-26  191.039871  50.38  51.9300  50.300       48695  51.815  8188776   
2022-08-29  187.767002  49.95  50.8650  49.930       48971  50.270  7306766   
2022-08-30  185.039434  48.79  49.7600  48.585       33010  49.650  5234435   
2022-08-31  184.087820  49.69  50.3250  49.530       35040  49.790  6156630   
2022-09-01  180.374139  49.22  49.3300  48.680       38620  49.150  5232575   
...                ...    ...      ...     ...         ...     ...      ...   
2024-02-21  197.569559  39.28  39.6150  39.120       18689  39.290  3192627   
2024-02-22  199.066619  39.77  39.8700  39.455       21611  39.810  3510076   
2024-02-23  199.850984  39.90  40.1900  39.730       20980  40.110  6138575   
2024-02-26  200.939981  39.55  39.8607  39.540       24944  39.670  7433109   
2024-02-27  203.566217  40.09  40.2000  40.015       23211  40.080  7255528   

                       
                 vwap  
2022-08-26  50.675660  
2022-08-29  50.371550  
2022-08-30  48.894582  
2022-08-31  49.787465  
2022-09-01  48.959277  
...               ...  
2024-02-21  39.293091  
2024-02-22  39.681310  
2024-02-23  39.897140  
2024-02-26  39.616831  
2024-02-27  40.099680  

[377 rows x 70 columns]

In [17]:
# Create a dataframe for closing prices
ticker_close = pd.DataFrame()
ticker_close['SPY Close'] = ticker_portfolio['SPY']['close']
ticker_close['EWA Close'] = ticker_portfolio['EWA']['close']
ticker_close['EWC Close'] = ticker_portfolio['EWC']['close']
ticker_close['EWJ Close'] = ticker_portfolio['EWJ']['close']
ticker_close['EWU Close'] = ticker_portfolio['EWU']['close']
ticker_close['EWZ Close'] = ticker_portfolio['EWZ']['close']
ticker_close['EWW Close'] = ticker_portfolio['EWW']['close']
ticker_close['EZU Close'] = ticker_portfolio['EZU']['close']
ticker_close['IWM Close'] = ticker_portfolio['EWA']['close']
ticker_close['MCHI Close'] = ticker_portfolio['MCHI']['close']
ticker_close = ticker_close.dropna()
ticker_close

,SPY Close,EWA Close,EWC Close,EWJ Close,EWU Close,EWZ Close,EWW Close,EZU Close,IWM Close,MCHI Close
2022-08-26,405.31,22.44,34.96,54.44,29.76,32.01,46.54,35.59,22.44,50.38
2022-08-29,402.63,22.36,34.98,53.89,29.73,32.29,45.92,35.71,22.36,49.95
2022-08-30,398.21,22.05,34.13,53.83,29.29,31.22,45.17,35.60,22.05,48.79
2022-08-31,395.18,21.91,33.69,53.59,28.91,30.52,44.25,35.33,21.91,49.69
2022-09-01,396.42,21.73,33.36,52.95,28.42,30.46,44.68,34.80,21.73,49.22
...,...,...,...,...,...,...,...,...,...,...
2024-02-21,497.21,23.63,36.49,67.79,32.65,33.69,67.12,48.31,23.63,39.28
2024-02-22,507.50,23.86,36.84,68.90,32.89,33.49,67.20,48.98,23.86,39.77
2024-02-23,507.85,23.91,36.99,69.01,33.01,33.12,66.53,49.01,23.91,39.90
2024-02-26,505.99,23.83,36.85,69.00,32.91,33.29,66.17,48.99,23.83,39.55


In [18]:
# Create dataframe for training on historical data (X features)
historical_df = ticker_close['SPY Close']
historical_df

2022-08-26    405.31
2022-08-29    402.63
2022-08-30    398.21
2022-08-31    395.18
2022-09-01    396.42
               ...  
2024-02-21    497.21
2024-02-22    507.50
2024-02-23    507.85
2024-02-26    505.99
2024-02-27    506.93
Name: SPY Close, Length: 377, dtype: float64

In [19]:
# Determine correlation between SPY and ETFs
ticker_close[[
    'SPY Close','EWA Close','EWC Close','EWJ Close','EWU Close',
    'EWZ Close','EWW Close', 'EZU Close','IWM Close','MCHI Close']].corr().style.background_gradient()


,SPY Close,EWA Close,EWC Close,EWJ Close,EWU Close,EWZ Close,EWW Close,EZU Close,IWM Close,MCHI Close
SPY Close,1.000000,0.452482,0.727259,0.938632,0.608213,0.632827,0.864371,0.765161,0.452482,-0.453632
EWA Close,0.452482,1.000000,0.864396,0.549356,0.775362,0.047114,0.607527,0.729749,1.000000,0.327814
EWC Close,0.727259,0.864396,1.000000,0.728529,0.748215,0.391995,0.750762,0.766842,0.864396,-0.004942
EWJ Close,0.938632,0.549356,0.728529,1.000000,0.780355,0.444627,0.929413,0.892631,0.549356,-0.256404
EWU Close,0.608213,0.775362,0.748215,0.780355,1.000000,0.014665,0.824091,0.947961,0.775362,0.239754
EWZ Close,0.632827,0.047114,0.391995,0.444627,0.014665,1.000000,0.410520,0.169822,0.047114,-0.610791
EWW Close,0.864371,0.607527,0.750762,0.929413,0.824091,0.410520,1.000000,0.941855,0.607527,-0.217799
EZU Close,0.765161,0.729749,0.766842,0.892631,0.947961,0.169822,0.941855,1.000000,0.729749,0.034724
IWM Close,0.452482,1.000000,0.864396,0.549356,0.775362,0.047114,0.607527,0.729749,1.000000,0.327814
MCHI Close,-0.453632,0.327814,-0.004942,-0.256404,0.239754,-0.610791,-0.217799,0.034724,0.327814,1.000000


In [20]:
# View ticker correlation in dataframe
corr_df = ticker_close[[
    'SPY Close','EWA Close','EWC Close','EWJ Close','EWU Close',
    'EWZ Close','EWW Close', 'EZU Close','IWM Close','MCHI Close']].corr()
print(corr_df)

            SPY Close  EWA Close  EWC Close  EWJ Close  EWU Close  EWZ Close  \
SPY Close    1.000000   0.452482   0.727259   0.938632   0.608213   0.632827   
EWA Close    0.452482   1.000000   0.864396   0.549356   0.775362   0.047114   
EWC Close    0.727259   0.864396   1.000000   0.728529   0.748215   0.391995   
EWJ Close    0.938632   0.549356   0.728529   1.000000   0.780355   0.444627   
EWU Close    0.608213   0.775362   0.748215   0.780355   1.000000   0.014665   
EWZ Close    0.632827   0.047114   0.391995   0.444627   0.014665   1.000000   
EWW Close    0.864371   0.607527   0.750762   0.929413   0.824091   0.410520   
EZU Close    0.765161   0.729749   0.766842   0.892631   0.947961   0.169822   
IWM Close    0.452482   1.000000   0.864396   0.549356   0.775362   0.047114   
MCHI Close  -0.453632   0.327814  -0.004942  -0.256404   0.239754  -0.610791   

            EWW Close  EZU Close  IWM Close  MCHI Close  
SPY Close    0.864371   0.765161   0.452482   -0.453632  
EWA

In [21]:
# Drop unnecessary columns
funds_df_2 = funds_df.drop(columns=['4 WEEKS BANK DISCOUNT','4 WEEKS COUPON EQUIVALENT',
                                           '8 WEEKS BANK DISCOUNT', '8 WEEKS COUPON EQUIVALENT',
                                           '13 WEEKS BANK DISCOUNT', '13 WEEKS COUPON EQUIVALENT',
                                           '17 WEEKS BANK DISCOUNT', '17 WEEKS COUPON EQUIVALENT',
                                           '26 WEEKS BANK DISCOUNT', '26 WEEKS COUPON EQUIVALENT', 
                                            '52 WEEKS COUPON EQUIVALENT'])
funds_df_2

,52 WEEKS BANK DISCOUNT
Date,
2023-12-29,4.55
2023-12-28,4.58
2023-12-27,4.54
2023-12-26,4.58
2023-12-22,4.62
...,...
2022-10-25,4.40
2022-10-24,4.40
2022-10-21,4.38


In [22]:
# Rename '52 WEEKS' column
funds_df_2['Fed Interest Rates'] = funds_df_2['52 WEEKS BANK DISCOUNT']
funds_df_2 = funds_df_2.drop(columns=['52 WEEKS BANK DISCOUNT'])
funds_df_2

,Fed Interest Rates
Date,
2023-12-29,4.55
2023-12-28,4.58
2023-12-27,4.54
2023-12-26,4.58
2023-12-22,4.62
...,...
2022-10-25,4.40
2022-10-24,4.40
2022-10-21,4.38


In [23]:
# Create a list to hold correlated etfs and respective symbols
# Create for loop to append highest correlated etfs in respective list
correlated_etfs = []
symbols = []
for index,df in corr_df.iterrows():
    if index == 'SPY Close':
        continue 
    if df['SPY Close'] >= .67:
        correlated_etfs.append(df['SPY Close'])
        symbols.append(index)
       
print(correlated_etfs)
print(symbols)

[0.7272593853971779, 0.9386316174495382, 0.8643706907113567, 0.7651609230428579]
['EWC Close', 'EWJ Close', 'EWW Close', 'EZU Close']


In [24]:
# Set trading parameters
symbol_df = ticker_close[symbols].copy() 
symbol_df['EWJ Returns'] = symbol_df['EWJ Close'].pct_change()
symbol_df['EWW Returns'] = symbol_df['EWW Close'].pct_change()
symbol_df['EZU Returns'] = symbol_df['EZU Close'].pct_change()
symbol_df['EWC Returns'] = symbol_df['EWC Close'].pct_change()
symbol_df = symbol_df.dropna()
symbol_df

,EWC Close,EWJ Close,EWW Close,EZU Close,EWJ Returns,EWW Returns,EZU Returns,EWC Returns
2022-08-29,34.98,53.89,45.92,35.71,-0.010103,-0.013322,0.003372,0.000572
2022-08-30,34.13,53.83,45.17,35.60,-0.001113,-0.016333,-0.003080,-0.024300
2022-08-31,33.69,53.59,44.25,35.33,-0.004458,-0.020368,-0.007584,-0.012892
2022-09-01,33.36,52.95,44.68,34.80,-0.011943,0.009718,-0.015001,-0.009795
2022-09-02,33.66,52.37,45.58,34.46,-0.010954,0.020143,-0.009770,0.008993
...,...,...,...,...,...,...,...,...
2024-02-21,36.49,67.79,67.12,48.31,-0.002061,-0.002823,0.005202,-0.001368
2024-02-22,36.84,68.90,67.20,48.98,0.016374,0.001192,0.013869,0.009592
2024-02-23,36.99,69.01,66.53,49.01,0.001597,-0.009970,0.000612,0.004072
2024-02-26,36.85,69.00,66.17,48.99,-0.000145,-0.005411,-0.000408,-0.003785


In [25]:
# Save a new copy of symbol_df into features_df and begin adding more features
features_df = symbol_df.copy()
features_df

,EWC Close,EWJ Close,EWW Close,EZU Close,EWJ Returns,EWW Returns,EZU Returns,EWC Returns
2022-08-29,34.98,53.89,45.92,35.71,-0.010103,-0.013322,0.003372,0.000572
2022-08-30,34.13,53.83,45.17,35.60,-0.001113,-0.016333,-0.003080,-0.024300
2022-08-31,33.69,53.59,44.25,35.33,-0.004458,-0.020368,-0.007584,-0.012892
2022-09-01,33.36,52.95,44.68,34.80,-0.011943,0.009718,-0.015001,-0.009795
2022-09-02,33.66,52.37,45.58,34.46,-0.010954,0.020143,-0.009770,0.008993
...,...,...,...,...,...,...,...,...
2024-02-21,36.49,67.79,67.12,48.31,-0.002061,-0.002823,0.005202,-0.001368
2024-02-22,36.84,68.90,67.20,48.98,0.016374,0.001192,0.013869,0.009592
2024-02-23,36.99,69.01,66.53,49.01,0.001597,-0.009970,0.000612,0.004072
2024-02-26,36.85,69.00,66.17,48.99,-0.000145,-0.005411,-0.000408,-0.003785


In [26]:
# Add SPY closing price daily returns to features dataframe
features_df['SPY Returns'] = ticker_close['SPY Close'].pct_change()
features_df

,EWC Close,EWJ Close,EWW Close,EZU Close,EWJ Returns,EWW Returns,EZU Returns,EWC Returns,SPY Returns
2022-08-29,34.98,53.89,45.92,35.71,-0.010103,-0.013322,0.003372,0.000572,-0.006612
2022-08-30,34.13,53.83,45.17,35.60,-0.001113,-0.016333,-0.003080,-0.024300,-0.010978
2022-08-31,33.69,53.59,44.25,35.33,-0.004458,-0.020368,-0.007584,-0.012892,-0.007609
2022-09-01,33.36,52.95,44.68,34.80,-0.011943,0.009718,-0.015001,-0.009795,0.003138
2022-09-02,33.66,52.37,45.58,34.46,-0.010954,0.020143,-0.009770,0.008993,-0.010544
...,...,...,...,...,...,...,...,...,...
2024-02-21,36.49,67.79,67.12,48.31,-0.002061,-0.002823,0.005202,-0.001368,0.000906
2024-02-22,36.84,68.90,67.20,48.98,0.016374,0.001192,0.013869,0.009592,0.020695
2024-02-23,36.99,69.01,66.53,49.01,0.001597,-0.009970,0.000612,0.004072,0.000690
2024-02-26,36.85,69.00,66.17,48.99,-0.000145,-0.005411,-0.000408,-0.003785,-0.003662


In [27]:
# Incorporate federal funds rate DF as a feature in features_df
features_df['Fed Funds Rate'] = funds_df_2
features_df = features_df.dropna()
features_df

,EWC Close,EWJ Close,EWW Close,EZU Close,EWJ Returns,EWW Returns,EZU Returns,EWC Returns,SPY Returns,Fed Funds Rate
2022-10-19,31.33,48.48,46.02,33.48,-0.006150,-0.005403,-0.014715,-0.007602,-0.007086,4.39
2022-10-20,31.17,48.44,46.28,33.49,-0.000825,0.005650,0.000299,-0.005107,-0.008385,4.44
2022-10-21,31.96,49.29,47.43,34.16,0.017547,0.024849,0.020006,0.025345,0.024301,4.38
2022-10-24,31.89,48.97,48.06,34.47,-0.006492,0.013283,0.009075,-0.002190,0.012237,4.40
2022-10-25,32.43,50.09,49.04,35.33,0.022871,0.020391,0.024949,0.016933,0.015969,4.40
...,...,...,...,...,...,...,...,...,...,...
2023-12-22,36.43,63.33,67.81,47.23,0.003168,-0.000590,-0.001691,0.007467,0.002010,4.62
2023-12-26,36.83,63.31,68.31,47.57,-0.000316,0.007374,0.007199,0.010980,0.004223,4.58
2023-12-27,36.87,63.66,68.46,47.81,0.005528,0.002196,0.005045,0.001086,0.001808,4.54
2023-12-28,36.70,64.04,67.96,47.41,0.005969,-0.007304,-0.008366,-0.004611,0.000378,4.58


In [28]:
# Rearrange columns
features_df = features_df.iloc[:,[0,1,2,3,4,5,6,7,9,8]]
features_df

,EWC Close,EWJ Close,EWW Close,EZU Close,EWJ Returns,EWW Returns,EZU Returns,EWC Returns,Fed Funds Rate,SPY Returns
2022-10-19,31.33,48.48,46.02,33.48,-0.006150,-0.005403,-0.014715,-0.007602,4.39,-0.007086
2022-10-20,31.17,48.44,46.28,33.49,-0.000825,0.005650,0.000299,-0.005107,4.44,-0.008385
2022-10-21,31.96,49.29,47.43,34.16,0.017547,0.024849,0.020006,0.025345,4.38,0.024301
2022-10-24,31.89,48.97,48.06,34.47,-0.006492,0.013283,0.009075,-0.002190,4.40,0.012237
2022-10-25,32.43,50.09,49.04,35.33,0.022871,0.020391,0.024949,0.016933,4.40,0.015969
...,...,...,...,...,...,...,...,...,...,...
2023-12-22,36.43,63.33,67.81,47.23,0.003168,-0.000590,-0.001691,0.007467,4.62,0.002010
2023-12-26,36.83,63.31,68.31,47.57,-0.000316,0.007374,0.007199,0.010980,4.58,0.004223
2023-12-27,36.87,63.66,68.46,47.81,0.005528,0.002196,0.005045,0.001086,4.54,0.001808
2023-12-28,36.70,64.04,67.96,47.41,0.005969,-0.007304,-0.008366,-0.004611,4.58,0.000378


In [29]:
# Add "y" column (SPY historical closing price) for testing neural models
features_df['y'] = 0
features_df

,EWC Close,EWJ Close,EWW Close,EZU Close,EWJ Returns,EWW Returns,EZU Returns,EWC Returns,Fed Funds Rate,SPY Returns,y
2022-10-19,31.33,48.48,46.02,33.48,-0.006150,-0.005403,-0.014715,-0.007602,4.39,-0.007086,0
2022-10-20,31.17,48.44,46.28,33.49,-0.000825,0.005650,0.000299,-0.005107,4.44,-0.008385,0
2022-10-21,31.96,49.29,47.43,34.16,0.017547,0.024849,0.020006,0.025345,4.38,0.024301,0
2022-10-24,31.89,48.97,48.06,34.47,-0.006492,0.013283,0.009075,-0.002190,4.40,0.012237,0
2022-10-25,32.43,50.09,49.04,35.33,0.022871,0.020391,0.024949,0.016933,4.40,0.015969,0
...,...,...,...,...,...,...,...,...,...,...,...
2023-12-22,36.43,63.33,67.81,47.23,0.003168,-0.000590,-0.001691,0.007467,4.62,0.002010,0
2023-12-26,36.83,63.31,68.31,47.57,-0.000316,0.007374,0.007199,0.010980,4.58,0.004223,0
2023-12-27,36.87,63.66,68.46,47.81,0.005528,0.002196,0.005045,0.001086,4.54,0.001808,0
2023-12-28,36.70,64.04,67.96,47.41,0.005969,-0.007304,-0.008366,-0.004611,4.58,0.000378,0


In [88]:
# Set trading parameters
features_df.loc[features_df['SPY Returns'] > 0.004,'y'] = 1
features_df.loc[features_df['SPY Returns'] < -0.002,'y'] = -1
features_df

,EWC Close,EWJ Close,EWW Close,EZU Close,EWJ Returns,EWW Returns,EZU Returns,EWC Returns,Fed Funds Rate,SPY Returns,y
2022-10-19,31.33,48.48,46.02,33.48,-0.006150,-0.005403,-0.014715,-0.007602,4.39,-0.007086,-1
2022-10-20,31.17,48.44,46.28,33.49,-0.000825,0.005650,0.000299,-0.005107,4.44,-0.008385,-1
2022-10-21,31.96,49.29,47.43,34.16,0.017547,0.024849,0.020006,0.025345,4.38,0.024301,1
2022-10-24,31.89,48.97,48.06,34.47,-0.006492,0.013283,0.009075,-0.002190,4.40,0.012237,1
2022-10-25,32.43,50.09,49.04,35.33,0.022871,0.020391,0.024949,0.016933,4.40,0.015969,1
...,...,...,...,...,...,...,...,...,...,...,...
2023-12-22,36.43,63.33,67.81,47.23,0.003168,-0.000590,-0.001691,0.007467,4.62,0.002010,1
2023-12-26,36.83,63.31,68.31,47.57,-0.000316,0.007374,0.007199,0.010980,4.58,0.004223,1
2023-12-27,36.87,63.66,68.46,47.81,0.005528,0.002196,0.005045,0.001086,4.54,0.001808,0
2023-12-28,36.70,64.04,67.96,47.41,0.005969,-0.007304,-0.008366,-0.004611,4.58,0.000378,0


In [116]:
# Set X (these are the initial features for neural network)
# Create a shift (lag) to verify effect of correlation on SPY Returns 
X = features_df[['EWC Close','EWJ Close','EWW Close','EZU Close',
                 'EWJ Returns','EWW Returns','EZU Returns','EWC Returns',
                 'Fed Funds Rate','SPY Returns']].shift().dropna().copy()
X.head(3)

,EWC Close,EWJ Close,EWW Close,EZU Close,EWJ Returns,EWW Returns,EZU Returns,EWC Returns,Fed Funds Rate,SPY Returns
2022-10-20,31.33,48.48,46.02,33.48,-0.006150,-0.005403,-0.014715,-0.007602,4.39,-0.007086
2022-10-21,31.17,48.44,46.28,33.49,-0.000825,0.005650,0.000299,-0.005107,4.44,-0.008385
2022-10-24,31.96,49.29,47.43,34.16,0.017547,0.024849,0.020006,0.025345,4.38,0.024301


In [117]:
y = features_df['y']
y = y.iloc[1:]
y

2022-10-20   -1
2022-10-21    1
2022-10-24    1
2022-10-25    1
2022-10-26   -1
             ..
2023-12-22    1
2023-12-26    1
2023-12-27    0
2023-12-28    0
2023-12-29   -1
Name: y, Length: 298, dtype: int64

In [118]:
# import libraries (potentially remove this block of code if redundant)
import pandas as pd
from pathlib import Path
import tensorflow as tf
from tensorflow.keras.layers import Dense
from tensorflow.keras.models import Sequential
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
# Ignore warnings
import warnings
warnings.filterwarnings('ignore')
%matplotlib inline

In [119]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)

In [120]:
X_scaler = StandardScaler()
X_scaler.fit(X_train)
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [121]:
number_inputs = 10
number_hidden_nodes = 5
neuron = Sequential()
neuron.add(Dense(units=number_hidden_nodes, input_dim=number_inputs, activation="relu"))
# Add the output layer to the model specifying the number of output neurons and activation function
neuron.add(Dense(1, activation="Softmax"))

In [122]:
neuron.summary()

Model: "sequential_17"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_34 (Dense)            (None, 5)                 55        
                                                                 
 dense_35 (Dense)            (None, 1)                 6         
                                                                 
Total params: 61 (244.00 Byte)
Trainable params: 61 (244.00 Byte)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [123]:
# Compile the Sequential model
neuron.compile(loss="CategoricalCrossentropy", optimizer="adam", metrics=["accuracy"])

In [124]:
# Fit the model using 100 epochs and the training data
model = neuron.fit(X_train_scaled, y_train, epochs=25)

Epoch 1/25
7/7 [==============================] - 2s 9ms/step - loss: 0.0000e+00 - accuracy: 0.4305
Epoch 2/25
7/7 [==============================] - 0s 15ms/step - loss: 0.0000e+00 - accuracy: 0.4305
Epoch 3/25
7/7 [==============================] - 0s 11ms/step - loss: 0.0000e+00 - accuracy: 0.4305
Epoch 4/25
7/7 [==============================] - 0s 8ms/step - loss: 0.0000e+00 - accuracy: 0.4305
Epoch 5/25
7/7 [==============================] - 0s 17ms/step - loss: 0.0000e+00 - accuracy: 0.4305
Epoch 6/25
7/7 [==============================] - 0s 12ms/step - loss: 0.0000e+00 - accuracy: 0.4305
Epoch 7/25
7/7 [==============================] - 0s 8ms/step - loss: 0.0000e+00 - accuracy: 0.4305
Epoch 8/25
7/7 [==============================] - 0s 7ms/step - loss: 0.0000e+00 - accuracy: 0.4305
Epoch 9/25
7/7 [==============================] - 0s 14ms/step - loss: 0.0000e+00 - accuracy: 0.4305
Epoch 10/25
7/7 [==============================] - 0s 14ms/step - loss: 0.0000e+00 - accuracy: 

In [ ]:
# Create a DataFrame using the model history and an index parameter
model_plot = pd.DataFrame(model.history, index=range(1, len(model.history["loss"]) + 1))
# Vizualize the model plot where the y-axis displays the loss metric
model_plot.plot(y="loss")

In [ ]:
# Vizualize the model plot where the y-axis displays the accuracy metric
model_plot.plot(y="accuracy")

In [ ]:
# Evaluate the model loss and accuracy metrics using the evaluate method and the test data
model_loss, model_accuracy = neuron.evaluate(X_test_scaled, y_test, verbose=2)
# Display the evaluation results
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")